In [ ]:
# Linking with google drive
# simple upload to github, manual here: https://medium.com/analytics-vidhya/how-to-use-google-colab-with-github-via-google-drive-68efb23a42d

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd /content/drive/My Drive/time_series_analysis/time_series_prediction

/content/drive/My Drive/time_series_analysis/time_series_prediction


In [ ]:
#!git clone https://{git_token}@github.com/{username}/{repository}

In [ ]:
# trying to import python module as a separate script

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/time_series_analysis/time_series_prediction/dataprocessing/')
sys.path.append('/content/drive/MyDrive/time_series_analysis/time_series_prediction/models/')

In [14]:
import datapreparation
import baselines
from matplotlib import pyplot as plt

In [4]:
%load utils/plotting.ipynb

In [5]:
%load models/baselines.ipynb

In [ ]:
#%load time_series_prediction/models/baselines.py

In [ ]:
#%load time_series_prediction/dataprocessing/datapreparation.py

In [6]:
import pandas as pd

In [7]:
sales_data = pd.read_csv("/content/sales.csv",   header = 0, sep = ";")
categories = pd.read_csv("/content/categories.csv",   header = 0, sep = ";")
traffic = pd.read_csv("/content/traffic.csv",   header = 0, sep = ";")

In [8]:
traffic['DateTime'] = traffic['week_starting_date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
traffic.drop("week_starting_date", axis = 1, inplace = True)

traffic = traffic.set_index(["product_id", "DateTime"])
categories = categories.set_index(["product_id"])

In [9]:
# extended only to global max date horizon
# check nulls here because looks strange
sales_df_combined_right = datapreparation.prepare_sales_data(sales_data, categories, traffic, True, True, [1,2,3])

In [11]:
sample = sales_df_combined_right.loc[[2658, 1308]]

In [ ]:
# for i in all_items:
#   print("*******************")
#   print(i)
#   tmp_dataset = sales_df_combined_right.loc[i]
#   #n_splits_tmp = int(np.floor((tmp_dataset.shape[0] - 3)/2))
#   try:
#     tscv = TimeSeriesSplit(n_splits=5, test_size = 3)
#     for tr_index, val_index in tscv.split(tmp_dataset):
#       print (tr_index, val_index)
#       X_tr, X_val = tmp_dataset[["sales"]].iloc[tr_index], tmp_dataset[["sales"]].iloc[val_index]
#       print(X_tr)
#       print(X_val)
#       forecast = get_naive_forecast(X_tr.sales, horizon = 3)
#       print(forecast)
#       print(mean_absolute_percentage_error(X_val.sales, forecast))
#   except Exception as e:
#     print(e)

### Git commands

In [ ]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Master_exploration.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	utils/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add .

In [ ]:
!git config --global user.email "katarzyna.staniewicz0504@gmail.com"

In [ ]:
!git commit -m 'Updates'

[master b349b6a] Updates
 2 files changed, 2 insertions(+), 1 deletion(-)
 rewrite Master_exploration.ipynb (93%)
 create mode 100644 utils/plotting.ipynb


In [ ]:
!git push

Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 3.14 KiB | 535.00 KiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/katarzynasornat/time_series_prediction
   009964d..b349b6a  master -> master


In [16]:
model_naive = baselines.NaiveForecast(horizon=3)

In [ ]:
model_drift = NaiveDrift(horizon=3)

NameError: ignored

In [ ]:
drift_forecast = model_drift.fit(sales_df_combined_right.loc[2658].sales[:-3]).predict(sales_df_combined_right.loc[2658].sales[-3:])
naive_forecast = model_naive.fit(sales_df_combined_right.loc[2658].sales[:-3]).predict(sales_df_combined_right.loc[2658].sales[-3:])

In [ ]:
forecasts = {'drift_foreast': drift_forecast, 'naive_forecast': naive_forecast}

In [ ]:
plot_original_and_forecasts(original_series = sales_df_combined_right.loc[2658].sales, dict_with_forecasts = forecasts)

In [ ]:
class TimeSeries(object):
  def __init__(self, series_with_numbers):
    self._time_col = time_col
    self._values = df
    self._product_id = df.product_id.iloc[0]

    self._time_indexed = pd.to_datetime(df[time_col])

    self.init_ts()
    self._how_many_nonzero_initaly()
  
  def init_ts(self):
    self._values.index = self._time_indexed
    self._ts = self._values.asfreq('W-Mon').sort_index(ascending = True).fillna(0)

  def _how_many_nonzero_initaly(self):
    self._non_zero_weeks = self._values.shape[0]
    self._non_zero_share = 100*round(self._non_zero_weeks/self._ts.shape[0],4)

  def plot_time_series(self, col_to_plot):
    self._ts[[col_to_plot]].plot(marker='o', title = f"product_id = {self._product_id}")